## 지하철데이터와 승하차인원데이터 결합하기

In [1]:
import pandas as pd

customers_subway = pd.read_csv("C:/data/subway_customers.csv")
customers_subway.head(5)

,사용일자,노선명,역ID,역명,승차총승객수,하차총승객수,등록일자
0,20200801,중앙선,오빈,249,239,20200804,NaN
1,20200801,중앙선,아신,467,595,20200804,NaN
2,20200801,중앙선,국수,520,598,20200804,NaN
3,20200801,2호선,한양대,3244,3022,20200804,NaN
4,20200801,2호선,왕십리(성동구청),11110,9769,20200804,NaN


In [2]:
# 승하차인원 9호선 데이터 수정
customers_subway['노선명'] = customers_subway['노선명'].replace({'9호선2~3단계' : '9호선'})

# 승하차인원 데이터 컬럼 이름 데이터에 맞추기
customers_subway = customers_subway[['사용일자','노선명','역ID','역명','승차총승객수']]
customers_subway = customers_subway.rename(columns = {'노선명' : '호선',
                                                     '역ID' : '전철역명',
                                                     '역명' : '승차승객수',
                                                     '승차총승객수' : '하차승객수'})

In [3]:
# 지하철데이터 수정
sub = pd.read_csv("C:/data/subway.csv")
sub = sub[['전철역명','호선','위도','경도']]

sub['호선'] = sub['호선'].replace(
            {'1' : '1호선', '2' : '2호선', '3' : '3호선',
             '4' : '4호선', '5' : '5호선', '6' : '6호선',
             '7' : '7호선', '8' : '8호선', '9' : '9호선',
             'B' : '분당선', 'S' : '신분당선', 'I' : '인천1호선',
             'I2' : '인천2호선', 'A' : '공항철도', 'K' : '경의중앙선',
             'UI' : '우이신설선', 'G' : '경춘선', 'SU' : '수인분당선',
             'U' : '의정부', 'KK' : '경강선', 'E' : '에버라인'})

sub.head(5)

,전철역명,호선,위도,경도
0,가락시장,8호선,37.492522,127.118234
1,가락시장,3호선,37.492522,127.118234
2,종로3가,5호선,37.571607,126.991806
3,종로3가,3호선,37.571607,126.991806
4,종로3가,1호선,37.571607,126.991806


In [4]:
# 지하철데이터, 승객수데이터의 전철역명에서 ([가-힣]) 제외하기
c = customers_subway.copy()

c['전철역명'] = c['전철역명'].str.replace('\([가-힣]+\)|\([a-zA-Z]|\)|\([가-힣]+[.]+[가-힣]+\)', '')
sub['전철역명'] = sub['전철역명'].str.replace('\([가-힣]+\)|\([a-zA-Z]|\)|\([가-힣]+[.]+[가-힣]+\)', '')

sub

,전철역명,호선,위도,경도
0,가락시장,8호선,37.492522,127.118234
1,가락시장,3호선,37.492522,127.118234
2,종로3가,5호선,37.571607,126.991806
3,종로3가,3호선,37.571607,126.991806
4,종로3가,1호선,37.571607,126.991806
...,...,...,...,...
924,동백,에버라인,37.269043,127.152716
925,초당,에버라인,37.260752,127.159443
926,삼가,에버라인,37.242115,127.168075
927,시청?용인대,에버라인,37.239151,127.178406


In [5]:
# 위경도값 없는 (서울시내) 지하철역 직접 추가

sub.iloc[258,2] = 37.649569       # 419민주묘지
sub.iloc[258,3] = 127.013695

sub.iloc[259,2] = 37.641561       # 가오리역
sub.iloc[259,3] = 127.016812

sub.iloc[260,2] = 37.634133        # 화계역
sub.iloc[260,3] = 127.017505

sub.iloc[314,2] = 37.662915        # 북한산우이역
sub.iloc[314,3] = 127.012799

sub.iloc[907,2] = 37.656066        # 솔밭공원역
sub.iloc[907,3] = 127.013281

sub.iloc[908,2] = 37.626932        # 삼양역
sub.iloc[908,3] = 127.018167

sub.iloc[909,2] = 37.621350        # 삼양사거리역
sub.iloc[909,3] = 127.020502

sub.iloc[910,2] = 37.620180        # 솔샘역
sub.iloc[910,3] = 127.013475

sub.iloc[911,2] = 37.612093        # 북한산보국문역
sub.iloc[911,3] = 127.008246

sub.iloc[912,2] = 37.602672        # 정릉역
sub.iloc[912,3] = 127.013495

In [6]:
sub[sub['위도'].isnull()]

,전철역명,호선,위도,경도
268,풍기,1호선,NaN,NaN
293,지평,경의중앙선,NaN,NaN
513,탕정,1호선,NaN,NaN
535,여주,경강선,NaN,NaN


In [7]:
sub[sub['경도'].isnull()]

,전철역명,호선,위도,경도
268,풍기,1호선,NaN,NaN
293,지평,경의중앙선,NaN,NaN
513,탕정,1호선,NaN,NaN
535,여주,경강선,NaN,NaN


In [8]:
# qgis 이용해 서울시내 지하철역만 새 테이블로 저장
sub_seoul = pd.read_csv("C:/data/now_used/full_seoul_subway.csv", encoding = 'UTF-8')

sub_seoul

,전철역명,호선,위도,경도
0,가락시장,8호선,37.492522,127.118234
1,가락시장,3호선,37.492522,127.118234
2,종로3가,5호선,37.571607,126.991806
3,종로3가,3호선,37.571607,126.991806
4,종로3가,1호선,37.571607,126.991806
...,...,...,...,...
578,삼양,우이신설선,37.626932,127.018167
579,삼양사거리,우이신설선,37.621350,127.020502
580,솔샘,우이신설선,37.620180,127.013475
581,북한산보국문,우이신설선,37.612093,127.008246


In [9]:
sub_seoul.iloc[219, 0] = '4.19민주묘지'
sub_seoul.iloc[219, :]

전철역명    4.19민주묘지
호선         우이신설선
위도       37.6496
경도       127.014
Name: 219, dtype: object

In [10]:
# c 테이블에 승차승객수 + 하차승객수 열 만들기
c['승하차총승객수'] = c['승차승객수'] + c['하차승객수']

c.head(5)

,사용일자,호선,전철역명,승차승객수,하차승객수,승하차총승객수
0,20200801,중앙선,오빈,249,239,488
1,20200801,중앙선,아신,467,595,1062
2,20200801,중앙선,국수,520,598,1118
3,20200801,2호선,한양대,3244,3022,6266
4,20200801,2호선,왕십리,11110,9769,20879


In [11]:
#  서울역 -> 서울 로 수정
c['전철역명'] = c['전철역명'].replace({'서울역' : '서울'})

c[c['전철역명'].str.endswith('역')]['전철역명'].unique()

array([], dtype=object)

In [12]:
c['호선'] = c['호선'].replace({'공항철도 1호선' : '공항철도', '중앙선' : '경의중앙선'})

c.head(5)

,사용일자,호선,전철역명,승차승객수,하차승객수,승하차총승객수
0,20200801,경의중앙선,오빈,249,239,488
1,20200801,경의중앙선,아신,467,595,1062
2,20200801,경의중앙선,국수,520,598,1118
3,20200801,2호선,한양대,3244,3022,6266
4,20200801,2호선,왕십리,11110,9769,20879


In [13]:
# 승하차총승객수 평일 하루 + 주말 하루 합쳐서 통합데이터 만들기
c_sample = c[(c['사용일자'] == 20200828) | (c['사용일자'] == 20200829)]

c_sample = c_sample[['사용일자','호선','전철역명','승하차총승객수']]

In [14]:
# 호선, 전철역명 기준으로 총승객수 구하기 = 사용일자 구분없이 승객수 합치기
import sqlite3

conn = sqlite3.connect(':memory:')
c = conn.cursor()

c_sample.to_sql('c_sample', conn, index = False)

c.execute('SELECT * FROM c_sample')
c.fetchone()

(20200828, '경부선', '당정', 9699)

In [15]:
c.execute("""SELECT 호선, 전철역명, SUM(승하차총승객수)
            FROM c_sample GROUP BY 호선, 전철역명""")
table = c.fetchall()

c.close()
conn.close()

customers = pd.DataFrame(table, columns = ['호선','전철역명','승하차총승객수'])

customers

,호선,전철역명,승하차총승객수
0,1호선,동대문,32168
1,1호선,동묘앞,24779
2,1호선,서울,91492
3,1호선,시청,42516
4,1호선,신설동,34993
...,...,...,...
589,장항선,봉명,2389
590,장항선,신창,3191
591,장항선,쌍용,4359
592,장항선,아산,6886


In [16]:
# 승하차승객수데이터 + 지하철데이터 합친 후 중복데이터 제거

df = pd.merge(sub_seoul, customers, on = ['호선','전철역명'], how = 'left').fillna(0)

df

,전철역명,호선,위도,경도,승하차총승객수
0,가락시장,8호선,37.492522,127.118234,20940.0
1,가락시장,3호선,37.492522,127.118234,22407.0
2,종로3가,5호선,37.571607,126.991806,27006.0
3,종로3가,3호선,37.571607,126.991806,23565.0
4,종로3가,1호선,37.571607,126.991806,59905.0
...,...,...,...,...,...
578,삼양,우이신설선,37.626932,127.018167,5349.0
579,삼양사거리,우이신설선,37.621350,127.020502,6462.0
580,솔샘,우이신설선,37.620180,127.013475,12832.0
581,북한산보국문,우이신설선,37.612093,127.008246,13836.0


In [17]:
df = df.drop_duplicates()
df = df.reset_index()
df = df.iloc[:, 1:]

df

,전철역명,호선,위도,경도,승하차총승객수
0,가락시장,8호선,37.492522,127.118234,20940.0
1,가락시장,3호선,37.492522,127.118234,22407.0
2,종로3가,5호선,37.571607,126.991806,27006.0
3,종로3가,3호선,37.571607,126.991806,23565.0
4,종로3가,1호선,37.571607,126.991806,59905.0
...,...,...,...,...,...
381,삼양,우이신설선,37.626932,127.018167,5349.0
382,삼양사거리,우이신설선,37.621350,127.020502,6462.0
383,솔샘,우이신설선,37.620180,127.013475,12832.0
384,북한산보국문,우이신설선,37.612093,127.008246,13836.0


In [18]:
# 환승역여부 컬럼 만들기
lst = []
for i in df[df['전철역명'].duplicated()]['전철역명']:
    lst.append(i)
lst

['가락시장',
 '종로3가',
 '종로3가',
 '오금',
 '동대문',
 '동대문역사문화공원',
 '동대문역사문화공원',
 '충무로',
 '서울',
 '서울',
 '신도림',
 '이촌',
 '왕십리',
 '왕십리',
 '왕십리',
 '시청',
 '교대',
 '신설동',
 '신설동',
 '까치산',
 '을지로3가',
 '홍대입구',
 '홍대입구',
 '광운대',
 '상봉',
 '상봉',
 '망우',
 '영등포구청',
 '신길',
 '을지로4가',
 '합정',
 '공덕',
 '공덕',
 '공덕',
 '삼각지',
 '약수',
 '신당',
 '동묘앞',
 '석계',
 '도봉산',
 '불광',
 '연신내',
 '회기',
 '회기',
 '디지털미디어시티',
 '디지털미디어시티',
 '선정릉',
 '종합운동장',
 '천호',
 '창동',
 '노원',
 '군자',
 '건대입구',
 '고속터미널',
 '고속터미널',
 '총신대입구',
 '대림',
 '가산디지털단지',
 '온수',
 '잠실',
 '복정',
 '당산',
 '여의도',
 '동작',
 '김포공항',
 '김포공항',
 '강남',
 '양재',
 '강남구청',
 '선릉',
 '효창공원앞',
 '태릉입구',
 '청량리',
 '청량리',
 '중랑',
 '충정로',
 '청구',
 '성신여대입구',
 '보문',
 '옥수',
 '도곡',
 '수서',
 '용산',
 '노량진',
 '서울',
 '신촌']

In [19]:
df['환승역여부'] = 0
for i in range(0, len(df) -1):
    if df['전철역명'][i] in lst:
        df['환승역여부'][i] = 1
        
df.head(5)
sub_copy = df.copy()

<ipython-input-19-56080433dc50>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['환승역여부'][i] = 1


## 승하차인원수 없는 지하철역 승객수 예측

* 행정동별 서울 생활인구수와 호선별 평균 이용승객수를 이용한다.

In [20]:
# 행정동별 서울생활인구 데이터 불러오기
people_seoul = pd.read_csv("C:/data/행정동별 서울생활인구(내국인).csv", encoding = 'ANSI')

people_seoul.head(5)

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20200923,0,11110515,14712.4525,556.2253,354.4283,512.0704,537.8768,526.8353,426.9263,...,410.0183,478.5760,613.2686,761.6991,740.4867,632.4125,547.3172,429.3045,314.6661,1134.7597
1,20200923,0,11110530,14645.3462,362.6159,161.1623,237.2480,392.7049,500.6554,543.4275,...,516.9224,731.7667,751.6547,601.4805,640.9400,567.2687,603.8864,467.2078,343.6354,1189.8916
2,20200923,0,11110540,4481.1216,70.4471,53.9792,187.4596,434.8234,171.7598,177.9516,...,105.3817,168.4759,176.5276,186.5206,164.8865,148.0386,180.9513,183.5713,146.5233,303.7295
3,20200923,0,11110550,15669.0514,396.3500,307.3244,438.9057,504.7364,410.8092,402.9325,...,476.9480,446.8077,628.5557,615.0809,826.9574,724.3836,690.9365,673.6508,412.8993,1419.1019
4,20200923,0,11110560,20288.1450,590.7363,427.2987,624.8066,610.5099,537.7352,424.6051,...,504.9341,531.4385,762.7111,776.0409,1006.4182,891.8553,1101.3865,878.2048,625.1792,1842.9204


In [21]:
people_seoul = people_seoul[(people_seoul['기준일ID'] == 20200829) | (people_seoul['기준일ID'] == 20200828)][['기준일ID','시간대구분','행정동코드','총생활인구수']]

people_seoul

,기준일ID,시간대구분,행정동코드,총생활인구수
254400,20200829,0,11110515,14534.2394
254401,20200829,0,11110530,14378.4765
254402,20200829,0,11110540,4159.9377
254403,20200829,0,11110550,15510.1211
254404,20200829,0,11110560,19668.5804
...,...,...,...,...
274747,20200828,23,11740650,26911.2596
274748,20200828,23,11740660,27136.7665
274749,20200828,23,11740685,54635.2149
274750,20200828,23,11740690,7630.9884


In [22]:
# 동별로 시간대별 총생활인구수 합계 만들기
import sqlite3

conn = sqlite3.connect(':memory:')
c = conn.cursor()

people_seoul.to_sql('people_seoul', conn, index = False)

c.execute('SELECT * FROM people_seoul')
c.fetchone()

(20200829, 0, 11110515, 14534.2394)

In [23]:
c.execute("""SELECT 기준일ID, 행정동코드, round(sum(총생활인구수),0)
            FROM people_seoul
            GROUP BY 기준일ID, 행정동코드""")
table = c.fetchall()

x_table = pd.DataFrame(table, columns = ['기준일','행정동코드','생활인구수합'])
x_table

,기준일,행정동코드,생활인구수합
0,20200828,11110515,371999.0
1,20200828,11110530,578541.0
2,20200828,11110540,132813.0
3,20200828,11110550,356099.0
4,20200828,11110560,468034.0
...,...,...,...
843,20200829,11740650,638828.0
844,20200829,11740660,646621.0
845,20200829,11740685,1281871.0
846,20200829,11740690,173389.0


In [24]:
# 평일 + 주말 인구수 합치기
x_table.to_sql('x_table', conn, index = False)

In [25]:
c.execute("""SELECT 행정동코드, sum(생활인구수합)
            FROM x_table
            GROUP BY 행정동코드""")
table = c.fetchall()

pp_seoul = pd.DataFrame(table, columns = ['행정동코드', '생활인구수합'])
pp_seoul

,행정동코드,생활인구수합
0,11110515,718182.0
1,11110530,976479.0
2,11110540,242331.0
3,11110550,721858.0
4,11110560,957209.0
...,...,...
419,11740650,1251850.0
420,11740660,1291578.0
421,11740685,2538302.0
422,11740690,341926.0


In [26]:
c.close()
conn.close()

In [27]:
# 행정동코드 + 생활인구수 데이터 합치기
emd_code = pd.read_csv("C:/data/행정동코드.csv", encoding = 'UTF-8')

df_seoul = pd.merge(pp_seoul, emd_code[['H_DNG_CD(행자부행정동코드)', 'DO_NM(시도명)',
       'CT_NM(시군구명)', 'H_DNG_NM(행정동명)']], left_on = '행정동코드', right_on = 'H_DNG_CD(행자부행정동코드)', how = 'left')[['행정동코드', '생활인구수합', 'DO_NM(시도명)',
       'CT_NM(시군구명)', 'H_DNG_NM(행정동명)']]

df_seoul = df_seoul.rename(columns = {'DO_NM(시도명)' : '시도', 'CT_NM(시군구명)' : '시군구', 'H_DNG_NM(행정동명)' : '행정동'})

df_seoul.head(5)

,행정동코드,생활인구수합,시도,시군구,행정동
0,11110515,718182.0,서울,종로구,청운효자동
1,11110530,976479.0,서울,종로구,사직동
2,11110540,242331.0,서울,종로구,삼청동
3,11110550,721858.0,서울,종로구,부암동
4,11110560,957209.0,서울,종로구,평창동


In [28]:
# 생활인구 주소 뽑아 csv로 떨어뜨린 후 geocoding 이용해 위경도 뽑아오기
pp = df_seoul.copy()

conn = sqlite3.connect(':memory:')
c = conn.cursor()

pp.to_sql('pp', conn, index = False)

c.execute('SELECT * FROM pp')
c.fetchone()

(11110515, 718182.0, '서울', '종로구', '청운효자동')

In [29]:
c.execute("""SELECT 행정동코드, 생활인구수합, 시도||' '||시군구||' '||행정동
            FROM pp""")
table = c.fetchall()

c.close()
conn.close()

pp_add = pd.DataFrame(table, columns = ['행정동코드','총생활인구수','주소'])

pp_add.to_csv("C:/data/now_used/pp_add.csv", index = False)

In [30]:
# geocoding에서 뽑아온 위경도데이터 불러오기
pp_add_4326 = pd.read_csv("C:/data/now_used/pp_add_4326.csv")

pp_add = pp_add_4326[['행정동코드','총생활인구','주소','X','Y']]
pp_add = pp_add.rename(columns = {'X' : '경도', 'Y' : '위도'})

pp_add.head(5)

,행정동코드,총생활인구,주소,경도,위도
0,11110515,29924.0,서울 종로구 청운효자동,126.970637,37.584012
1,11110530,40687.0,서울 종로구 사직동,126.965955,37.575412
2,11110540,10097.0,서울 종로구 삼청동,126.981006,37.590761
3,11110550,30077.0,서울 종로구 부암동,126.965585,37.594772
4,11110560,39884.0,서울 종로구 평창동,126.974496,37.613032


In [31]:
# 지하철 위경도 ~ 생활인구 위경도 가장 가까운 지역의 생활인구 뽑아오기
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 1)
clf.fit(pp_add[['위도','경도']], pp_add['총생활인구'])

x = sub_copy['위도'].tolist()
y = sub_copy['경도'].tolist()
z = []

for i, j in zip(x,y):
    z.append(clf.predict(np.array([[i,j]]))[0])

sub_copy['행정동생활인구'] = z
sub_copy

,전철역명,호선,위도,경도,승하차총승객수,환승역여부,행정동생활인구
0,가락시장,8호선,37.492522,127.118234,20940.0,1,70228.0
1,가락시장,3호선,37.492522,127.118234,22407.0,1,70228.0
2,종로3가,5호선,37.571607,126.991806,27006.0,1,27485.0
3,종로3가,3호선,37.571607,126.991806,23565.0,1,27485.0
4,종로3가,1호선,37.571607,126.991806,59905.0,1,27485.0
...,...,...,...,...,...,...,...
381,삼양,우이신설선,37.626932,127.018167,5349.0,0,48316.0
382,삼양사거리,우이신설선,37.621350,127.020502,6462.0,0,47151.0
383,솔샘,우이신설선,37.620180,127.013475,12832.0,0,49317.0
384,북한산보국문,우이신설선,37.612093,127.008246,13836.0,0,38769.0


In [32]:
# 호선평균승객수 구하기
import sqlite3

conn = sqlite3.connect(':memory:')
c = conn.cursor()

sub_copy.to_sql('sub_copy', conn, index = False)

c.execute("SELECT * FROM sub_copy")
c.fetchone()

('가락시장', '8호선', 37.492521999999994, 127.11823400000002, 20940.0, 1, 70228.0)

In [33]:
c.execute("""SELECT 호선, round(avg(승하차총승객수),2)
            FROM sub_copy
            GROUP BY 호선""")
table = c.fetchall()

c.close()
conn.close()

sub_avg = pd.DataFrame(table, columns = ['호선','호선별평균승객수'])
sub_real_final = pd.merge(sub_copy, sub_avg, on = ['호선'], how = 'left')

sub_real_final

,전철역명,호선,위도,경도,승하차총승객수,환승역여부,행정동생활인구,호선별평균승객수
0,가락시장,8호선,37.492522,127.118234,20940.0,1,70228.0,31369.00
1,가락시장,3호선,37.492522,127.118234,22407.0,1,70228.0,35314.15
2,종로3가,5호선,37.571607,126.991806,27006.0,1,27485.0,27001.98
3,종로3가,3호선,37.571607,126.991806,23565.0,1,27485.0,35314.15
4,종로3가,1호선,37.571607,126.991806,59905.0,1,27485.0,13731.74
...,...,...,...,...,...,...,...,...
381,삼양,우이신설선,37.626932,127.018167,5349.0,0,48316.0,8125.00
382,삼양사거리,우이신설선,37.621350,127.020502,6462.0,0,47151.0,8125.00
383,솔샘,우이신설선,37.620180,127.013475,12832.0,0,49317.0,8125.00
384,북한산보국문,우이신설선,37.612093,127.008246,13836.0,0,38769.0,8125.00


In [34]:
# 승객수가 0인 전철역과 가장 비슷한 유동인구수, 호선별평균승객수를 가진 역 3개를 가져와 승객수 평균내서 승객수컬럼에 추가
sub_copy = sub_real_final.copy()

sub_0 = sub_copy[sub_copy['승하차총승객수'] == 0]
sub_1 = sub_copy[sub_copy['승하차총승객수'] > 0]

In [35]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors = 3, weights = 'uniform')
regressor.fit(sub_1[['행정동생활인구','호선별평균승객수']], sub_1[['승하차총승객수']])

x = sub_0['행정동생활인구'].tolist()
y = sub_0['호선별평균승객수'].tolist()
z = []

for i,j in zip(x,y):
    z.append(round(regressor.predict(np.array([[i,j]]))[0][0],2))
    
sub_0['예측승객수'] = z
sub_0

<ipython-input-35-bf259a9fe0f7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_0['예측승객수'] = z


,전철역명,호선,위도,경도,승하차총승객수,환승역여부,행정동생활인구,호선별평균승객수,예측승객수
9,동대문역사문화공원,5호선,37.565138,127.007896,0.0,1,37821.0,27001.98,11852.67
10,동대문역사문화공원,2호선,37.565138,127.007896,0.0,1,37821.0,62734.14,65184.67
11,동대문역사문화공원,4호선,37.565138,127.007896,0.0,1,37821.0,42443.64,48868.00
18,신도림,1호선,37.508725,126.891295,0.0,1,68374.0,13731.74,29000.00
19,이촌,경의중앙선,37.522272,126.974345,0.0,1,55428.0,5194.77,9862.33
...,...,...,...,...,...,...,...,...,...
257,오류동,1호선,37.494526,126.845365,0.0,0,32861.0,13731.74,50637.67
258,역곡,1호선,37.485178,126.811502,0.0,0,44498.0,13731.74,35941.67
261,방학,1호선,37.667503,127.044273,0.0,0,38077.0,13731.74,25904.33
262,도봉,1호선,37.679563,127.045595,0.0,0,35080.0,13731.74,40242.67


In [36]:
sub_0 = sub_0[['전철역명','호선','위도','경도','환승역여부','예측승객수']]
sub_0 = sub_0.rename(columns = {'예측승객수' : '승하차총승객수'})

In [37]:
# sub_0 + sub_1 합치기
sub_final = pd.concat([sub_0, sub_1[['전철역명','호선','위도','경도','환승역여부','승하차총승객수']]])

sub_final

,전철역명,호선,위도,경도,환승역여부,승하차총승객수
9,동대문역사문화공원,5호선,37.565138,127.007896,1,11852.67
10,동대문역사문화공원,2호선,37.565138,127.007896,1,65184.67
11,동대문역사문화공원,4호선,37.565138,127.007896,1,48868.00
18,신도림,1호선,37.508725,126.891295,1,29000.00
19,이촌,경의중앙선,37.522272,126.974345,1,9862.33
...,...,...,...,...,...,...
381,삼양,우이신설선,37.626932,127.018167,0,5349.00
382,삼양사거리,우이신설선,37.621350,127.020502,0,6462.00
383,솔샘,우이신설선,37.620180,127.013475,0,12832.00
384,북한산보국문,우이신설선,37.612093,127.008246,0,13836.00


In [38]:
# 전철역별로 승객수 합치기
conn = sqlite3.connect(':memory:')
c = conn.cursor()

sub_final.to_sql('sub_final', conn, index = False)

c.execute("SELECT * FROM sub_final")
c.fetchone()

('동대문역사문화공원', '5호선', 37.565138, 127.007896, 1, 11852.67)

In [39]:
c.execute("""SELECT 전철역명, 위도, 경도, 환승역여부, sum(승하차총승객수)
            FROM sub_final
            GROUP BY 전철역명""")
table = c.fetchall()

sub_combined = pd.DataFrame(table, columns = ['전철역명','위도','경도','환승역여부','승하차총승객수'])

c.close()
conn.close()


sub_combined.to_csv("C:/data/now_used/sub_final.csv", index = False)

sub_combined

,전철역명,위도,경도,환승역여부,승하차총승객수
0,4.19민주묘지,37.649569,127.013695,0,6928.00
1,가락시장,37.492522,127.118234,1,43347.00
2,가산디지털단지,37.481072,126.882343,1,140991.33
3,가양,37.561391,126.854456,0,50458.00
4,가오리,37.641561,127.016812,0,10194.00
...,...,...,...,...,...
295,화랑대,37.620064,127.084689,0,23890.00
296,회기,37.589460,127.057583,1,116690.66
297,회현,37.558514,126.978246,0,53619.00
298,효창공원앞,37.539261,126.961351,1,37261.33


## 상가데이터와 지하철데이터 이용해 승객수점수 컬럼 생성

각 상가에서 가장 가까운 지하철역 3개를 뽑아 그 역의 승객수를 떨어진 거리만큼 차등 계산하여 승객수점수를 만든다.

### a * 1/x + b * 1/y + c * 1/z

- a: 가까운 역 1의 승객수
- b: 가까운 역 2의 승객수
- c: 가까운 역 3의 승객수
- x: 1까지의 거리
- y: 2까지의 거리
- z: 3까지의 거리

In [40]:
# 서울 상가 데이터, 건축물대장 데이터 불러와 결합

store = pd.read_csv("C:/data/SHOP_RENT_BULD.csv", sep = '|')
store_info = pd.read_csv("C:/data/상가건물정보.csv", sep = '|')

seoul = pd.merge(store_info, store, on = '건축물대장코드', how = 'inner')

seoul.head(5)

C:\python\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,6,13,31,32,33,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,상가코드_x,건축물대장코드,도시,구시군,구,읍면동,상세주소,대지구분코드,번,지,...,순번,상가코드_y,STD_YYYYMM,n,usage_area,cont_area,RENT,rent_ua,rent_ca,floor
0,S11110000000004,11110-10014,서울특별시,종로구,NaN,동숭동,NaN,0.0,1,114,...,0,S11110000000004,202006,1,15.5,15.50,750.0,48.387097,48.387097,1
1,S11110000000125,11110-100180009,서울특별시,종로구,NaN,동숭동,NaN,0.0,1,67,...,0,S11110000000125,202003,2,70.6,70.60,1200.0,16.997167,16.997167,1
2,S11110000000125,11110-100180009,서울특별시,종로구,NaN,동숭동,NaN,0.0,1,67,...,0,S11110000000125,202006,5,103.0,143.92,1355.0,13.155340,9.414953,1
3,S11110000000220,11110-100184402,서울특별시,종로구,NaN,관훈동,NaN,0.0,186,0,...,0,S11110000000220,202006,1,185.0,200.00,2200.0,11.891892,11.000000,1
4,S11110000000222,11110-100184422,서울특별시,종로구,NaN,종로5가,NaN,0.0,314,12,...,0,S11110000000222,202006,2,55.0,68.00,167.0,3.036364,2.455882,2


In [41]:
# 필요한 컬럼만 갖고오기

seoul = seoul[['상가코드_x','건축물대장코드','STD_YYYYMM','도시','구시군','읍면동','번','지','입주일','상가유형',
               '지하총층','지상총층','대지면적m2','건축면적m2','연면적m2','건폐율','용적률','건축구조',
               '높이m','주차대수','승강기수','지도좌표X','지도좌표Y','n','usage_area','cont_area','RENT',
               'rent_ua','rent_ca','floor']]

seoul = seoul.rename(columns = {'상가코드_x' : '상가코드', 'STD_YYYYMM' : '입력날짜',
                                '지도좌표X' : '경도', '지도좌표Y' : '위도'})

In [42]:
seoul = seoul[seoul['도시'] == '서울특별시']
seoul

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,승강기수,경도,위도,n,usage_area,cont_area,RENT,rent_ua,rent_ca,floor
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,1,127.002617,37.581579,1,15.50,15.50,750.0,48.387097,48.387097,1
1,S11110000000125,11110-100180009,202003,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,4,127.003184,37.582038,2,70.60,70.60,1200.0,16.997167,16.997167,1
2,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,4,127.003184,37.582038,5,103.00,143.92,1355.0,13.155340,9.414953,1
3,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,1,126.984489,37.574157,1,185.00,200.00,2200.0,11.891892,11.000000,1
4,S11110000000222,11110-100184422,202006,서울특별시,종로구,종로5가,314,12,2.00803e+07,근린상가,...,0,127.003198,37.569933,2,55.00,68.00,167.0,3.036364,2.455882,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31896,S11740000017545,11740-9917,202003,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,2,127.138576,37.536962,1,45.14,280.99,180.0,3.987594,0.640592,2
31897,S11740000017545,11740-9917,202006,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,2,127.138576,37.536962,3,227.91,300.00,890.0,3.905050,2.966667,1
31898,S11740000017545,11740-9917,202006,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,2,127.138576,37.536962,3,324.20,680.99,540.0,1.665638,0.792963,2
31899,S11740000017557,11740-9929,202003,서울특별시,강동구,고덕동,294,4,1.98803e+07,근린상가,...,0,127.158444,37.560665,1,26.00,26.00,40.0,1.538462,1.538462,1


In [43]:
# qgis 이용해 상가별 가까운 지하철역 3개 찾은 테이블 갖고오기
sanga_sub = pd.read_csv("C:/data/now_used/sanga_sub.csv")

sanga_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95703 entries, 0 to 95702
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   InputID   95703 non-null  object 
 1   TargetID  95703 non-null  object 
 2   Distance  95703 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.2+ MB


In [44]:
# 상가-지하철역 3개 데이터에 지하철역 승객수 컬럼 추가
sanga_ppl_sub = pd.merge(sanga_sub, sub_combined[['전철역명','승하차총승객수']],
                        left_on = 'TargetID', right_on = '전철역명', how = 'left')

sanga_ppl_sub

,InputID,TargetID,Distance,전철역명,승하차총승객수
0,S11110000000004,혜화,0.001082,혜화,67118.0
1,S11110000000004,한성대입구,0.007766,한성대입구,32374.0
2,S11110000000004,종로5가,0.010680,종로5가,57015.0
3,S11110000000125,혜화,0.001373,혜화,67118.0
4,S11110000000125,한성대입구,0.007102,한성대입구,32374.0
...,...,...,...,...,...
95698,S11740000017557,상일동,0.008899,상일동,34572.0
95699,S11740000017557,명일,0.017177,명일,20480.0
95700,S11740000017588,둔촌동,0.005279,둔촌동,20803.0
95701,S11740000017588,길동,0.005532,길동,21061.0


In [45]:
# sanga_ppl_sub에 1/(distance*1000) * 승하차총승객수
sanga_ppl_sub['승객수점수'] = round((1/(sanga_ppl_sub['Distance']*1000)) * sanga_ppl_sub['승하차총승객수'],3)

sanga_ppl_sub

,InputID,TargetID,Distance,전철역명,승하차총승객수,승객수점수
0,S11110000000004,혜화,0.001082,혜화,67118.0,62027.001
1,S11110000000004,한성대입구,0.007766,한성대입구,32374.0,4168.645
2,S11110000000004,종로5가,0.010680,종로5가,57015.0,5338.252
3,S11110000000125,혜화,0.001373,혜화,67118.0,48879.367
4,S11110000000125,한성대입구,0.007102,한성대입구,32374.0,4558.289
...,...,...,...,...,...,...
95698,S11740000017557,상일동,0.008899,상일동,34572.0,3884.754
95699,S11740000017557,명일,0.017177,명일,20480.0,1192.290
95700,S11740000017588,둔촌동,0.005279,둔촌동,20803.0,3940.769
95701,S11740000017588,길동,0.005532,길동,21061.0,3807.342


In [46]:
# 상가코드별로 승객수점수 합계 구하기
conn = sqlite3.connect(':memory:')
c = conn.cursor()

sanga_ppl_sub.to_sql('sanga_ppl_sub', conn, index = False)

c.execute('SELECT * FROM sanga_ppl_sub')
c.fetchone()

('S11110000000004', '혜화', 0.00108207714999, '혜화', 67118.0, 62027.001)

In [47]:
c.execute("""SELECT InputID, TargetID, Distance, sum(승객수점수)
            FROM sanga_ppl_sub
            GROUP BY InputID""")
table = c.fetchall()

c.close()
conn.close()

sanga_ppl_final = pd.DataFrame(table, columns = ['상가코드','전철역명','가까운지하철거리','승객수점수'])

sanga_ppl_final

,상가코드,전철역명,가까운지하철거리,승객수점수
0,S11110000000004,혜화,0.001082,71533.898
1,S11110000000125,혜화,0.001373,117064.164
2,S11110000000220,안국,0.002509,46623.003
3,S11110000000222,종로5가,0.001675,60471.742
4,S11110000000300,종로5가,0.003434,36064.496
...,...,...,...,...
19530,S11740000017467,길동,0.006166,22247.037
19531,S11740000017519,둔촌동,0.002683,13277.534
19532,S11740000017545,길동,0.001656,90212.844
19533,S11740000017557,고덕,0.007105,7948.266


In [48]:
# 승객수점수 테이블과 seoul 테이블 결합
seoul_ppl = pd.merge(seoul, sanga_ppl_final, on = '상가코드', how = 'inner')

seoul_ppl

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,n,usage_area,cont_area,RENT,rent_ua,rent_ca,floor,전철역명,가까운지하철거리,승객수점수
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,1,15.50,15.50,750.0,48.387097,48.387097,1,혜화,0.001082,71533.898
1,S11110000000125,11110-100180009,202003,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,2,70.60,70.60,1200.0,16.997167,16.997167,1,혜화,0.001373,117064.164
2,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,5,103.00,143.92,1355.0,13.155340,9.414953,1,혜화,0.001373,117064.164
3,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,1,185.00,200.00,2200.0,11.891892,11.000000,1,안국,0.002509,46623.003
4,S11110000000222,11110-100184422,202006,서울특별시,종로구,종로5가,314,12,2.00803e+07,근린상가,...,2,55.00,68.00,167.0,3.036364,2.455882,2,종로5가,0.001675,60471.742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31896,S11740000017545,11740-9917,202003,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,1,45.14,280.99,180.0,3.987594,0.640592,2,길동,0.001656,90212.844
31897,S11740000017545,11740-9917,202006,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,3,227.91,300.00,890.0,3.905050,2.966667,1,길동,0.001656,90212.844
31898,S11740000017545,11740-9917,202006,서울특별시,강동구,길동,413,1,2.00202e+07,기타상가,...,3,324.20,680.99,540.0,1.665638,0.792963,2,길동,0.001656,90212.844
31899,S11740000017557,11740-9929,202003,서울특별시,강동구,고덕동,294,4,1.98803e+07,근린상가,...,1,26.00,26.00,40.0,1.538462,1.538462,1,고덕,0.007105,7948.266


## 지번별 에너지 사용량, 본번별 평균 에너지 사용량 컬럼 생성

In [49]:
df = seoul_ppl.copy()
df = df[df['floor'] == 1]
df = df[df['입력날짜'] == 202006]

df

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,n,usage_area,cont_area,RENT,rent_ua,rent_ca,floor,전철역명,가까운지하철거리,승객수점수
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,1,15.50,15.50,750.0,48.387097,48.387097,1,혜화,0.001082,71533.898
2,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,5,103.00,143.92,1355.0,13.155340,9.414953,1,혜화,0.001373,117064.164
3,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,1,185.00,200.00,2200.0,11.891892,11.000000,1,안국,0.002509,46623.003
7,S11110000000359,11110-100187137,202006,서울특별시,종로구,옥인동,145,0,2.01001e+07,기타상가,...,1,34.42,42.91,155.0,4.503196,3.612212,1,경복궁,0.007578,21174.100
9,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2.01109e+07,근린상가,...,1,58.22,58.22,220.0,3.778770,3.778770,1,경복궁,0.004792,29040.490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31887,S11740000017337,11740-9676,202006,서울특별시,강동구,명일동,312,42,1.98006e+07,근린상가,...,1,41.19,41.19,430.0,10.439427,10.439427,1,명일,0.000730,33569.452
31888,S11740000017364,11740-9698,202006,서울특별시,강동구,명일동,326,11,1.9811e+07,근린상가,...,4,127.60,127.60,480.0,3.761755,3.761755,1,명일,0.002046,16942.142
31889,S11740000017459,11740-9824,202006,서울특별시,강동구,상일동,280,5,1.99301e+07,근린상가,...,5,336.68,363.60,695.0,2.064275,1.911441,1,상일동,0.008948,11505.322
31892,S11740000017467,11740-9843,202006,서울특별시,강동구,둔촌동,609,3,1.99312e+07,근린상가,...,8,291.07,321.87,1360.0,4.672416,4.225308,1,길동,0.006166,22247.037


In [50]:
# 지번별 에너지 사용량
elec06 = pd.read_csv("C:/data/now_used/지번별_전기_민간개방용구조_202006.csv", 
                     header = None,
                     names = ['사용년월','순번','시군구코드','법정동코드','시도명',
                                '시군구명','법정동명','대지구분코드','본번','부번',
                                '새주소대로로코드','새주소대로로명','새주소지상지하코드',
                                '새주소지상지하명','새주소본번','새주소부번','사용량'],
                     sep = '|', encoding = 'ANSI')

elec06_seoul = elec06[elec06['시도명'] == '서울특별시']

elec06_seoul = elec06_seoul.rename(columns = {'시군구명' : '구시군', '법정동명' : '읍면동', '본번' : '번', '부번' : '지'})

elec06_seoul.head(5)

,사용년월,순번,시군구코드,법정동코드,시도명,구시군,읍면동,대지구분코드,번,지,새주소대로로코드,새주소대로로명,새주소지상지하코드,새주소지상지하명,새주소본번,새주소부번,사용량
0,202006,1,11110,10100,서울특별시,종로구,청운동,0,3,100,1.111031e+11,창의문로,0.0,지상,119.0,0.0,1019
1,202006,1,11110,10100,서울특별시,종로구,청운동,0,4,20,1.111041e+11,자하문로36길,0.0,지상,40.0,0.0,3014
2,202006,1,11110,10100,서울특별시,종로구,청운동,0,5,0,1.111031e+11,자하문로,0.0,지상,152.0,0.0,1991
3,202006,1,11110,10100,서울특별시,종로구,청운동,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,1716
4,202006,1,11110,10100,서울특별시,종로구,청운동,0,6,3,1.111041e+11,자하문로35길,0.0,지상,21.0,0.0,171


In [51]:
df_elec = pd.merge(df, elec06_seoul[['구시군','읍면동','번','지','사용량']], on = ['구시군','읍면동','번','지'])
df_elec = df_elec.rename(columns = {'사용량' : '건물별전기사용량'})

len(df_elec)

9342

In [52]:
# 본번별 전기에너지 사용량 구하기import sqlite3
conn = sqlite3.connect(':memory:')

c = conn.cursor()

elec06_seoul.to_sql('elec06_seoul',conn,index = False)

c.execute('select * from elec06_seoul where 읍면동 = "동숭동"')
c.fetchone()

(202006,
 1,
 11110,
 16800,
 '서울특별시',
 '종로구',
 '동숭동',
 0,
 1,
 1,
 111104100075.0,
 '동숭길',
 0.0,
 '지상',
 122.0,
 6.0,
 9475)

In [53]:
c.execute('SELECT 시도명, 구시군, 읍면동, 번, ROUND(AVG(사용량),2) FROM elec06_seoul GROUP BY 구시군, 읍면동, 번')
table2 = c.fetchall()

df2 = pd.DataFrame(table2, columns = ['시도명','구시군','읍면동','번','본번별평균사용량'])

c.close()
conn.close()

df2

,시도명,구시군,읍면동,번,본번별평균사용량
0,서울특별시,강남구,개포동,12,339415.80
1,서울특별시,강남구,개포동,13,152321.33
2,서울특별시,강남구,개포동,14,121596.67
3,서울특별시,강남구,개포동,121,384.00
4,서울특별시,강남구,개포동,139,10459.00
...,...,...,...,...,...
39473,서울특별시,중랑구,중화동,387,9382.00
39474,서울특별시,중랑구,중화동,404,286.00
39475,서울특별시,중랑구,중화동,413,1827.00
39476,서울특별시,중랑구,중화동,429,1403.00


In [54]:
df3 = pd.merge(df_elec, df2[['구시군','읍면동','번','본번별평균사용량']], on = ['구시군','읍면동','번'], how = 'left')

df3.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,cont_area,RENT,rent_ua,rent_ca,floor,전철역명,가까운지하철거리,승객수점수,건물별전기사용량,본번별평균사용량
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,15.50,750.0,48.387097,48.387097,1,혜화,0.001082,71533.898,11937,15813.56
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,143.92,1355.0,13.155340,9.414953,1,혜화,0.001373,117064.164,57888,15813.56
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,200.00,2200.0,11.891892,11.000000,1,안국,0.002509,46623.003,3261,3261.00
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2.01109e+07,근린상가,...,58.22,220.0,3.778770,3.778770,1,경복궁,0.004792,29040.490,2650,2650.00
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2.0111e+07,근린상가,...,66.20,120.0,2.037351,1.812689,1,종로5가,0.003819,34219.916,209,390.00


In [55]:
# 가장 가까운 횡단보도까지의 거리 구하기 - 신뢰도 의심
seoul_data = df3.copy()

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from haversine import haversine #pip install haversine

cross = pd.read_csv("C:/data/crosswalk_seoul.csv")

cross = cross[['순번','위도','경도']]
cross.columns

clf = KNeighborsClassifier(n_neighbors = 1)
clf.fit(cross[['위도','경도']],cross['순번'])

x = seoul_data['위도'].tolist()
y = seoul_data['경도'].tolist()
z = []
t = []
for i,j in zip(x,y):
    z.append(clf.predict(np.array([[i,j]]))[0])     # 가까운 지하철역 이름
    x = clf.predict(np.array([[i,j]]))[0]
    a = [i,j]       # 각 상가의 위도, 경도
    b = cross[cross['순번'] == x][['위도','경도']].iloc[0].tolist()      # 가까운 지하철역 위도, 경도
    t.append(haversine(a, b))   # 위도, 경도 이용해서 거리 구하기(km)



seoul_data_copy = seoul_data.copy()

seoul_data_copy['횡단보도거리'] = t

seoul_data_copy

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,RENT,rent_ua,rent_ca,floor,전철역명,가까운지하철거리,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,750.0,48.387097,48.387097,1,혜화,0.001082,71533.898,11937,15813.56,0.045544
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,1355.0,13.155340,9.414953,1,혜화,0.001373,117064.164,57888,15813.56,0.096056
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,2200.0,11.891892,11.000000,1,안국,0.002509,46623.003,3261,3261.00,0.130889
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2.01109e+07,근린상가,...,220.0,3.778770,3.778770,1,경복궁,0.004792,29040.490,2650,2650.00,0.010823
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2.0111e+07,근린상가,...,120.0,2.037351,1.812689,1,종로5가,0.003819,34219.916,209,390.00,0.039419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9337,S11740000017328,11740-9659,202006,서울특별시,강동구,고덕동,645,4,1.98809e+07,근린상가,...,1690.0,2.156190,1.894980,1,고덕,0.004357,8921.374,2408,3264.43,0.016357
9338,S11740000017337,11740-9676,202006,서울특별시,강동구,명일동,312,42,1.98006e+07,근린상가,...,430.0,10.439427,10.439427,1,명일,0.000730,33569.452,18399,12269.25,0.023463
9339,S11740000017364,11740-9698,202006,서울특별시,강동구,명일동,326,11,1.9811e+07,근린상가,...,480.0,3.761755,3.761755,1,명일,0.002046,16942.142,14429,8845.75,0.097750
9340,S11740000017467,11740-9843,202006,서울특별시,강동구,둔촌동,609,3,1.99312e+07,근린상가,...,1360.0,4.672416,4.225308,1,길동,0.006166,22247.037,4695,9900.00,0.021561


In [56]:
seoul_data_copy['횡단보도거리'] = seoul_data_copy['횡단보도거리'] * 1000
seoul_data_copy = seoul_data_copy.rename(columns = {'횡단보도거리' : '횡단보도거리m'})

seoul_data_copy.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,RENT,rent_ua,rent_ca,floor,전철역명,가까운지하철거리,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리m
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,750.0,48.387097,48.387097,1,혜화,0.001082,71533.898,11937,15813.56,45.543627
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,1355.0,13.155340,9.414953,1,혜화,0.001373,117064.164,57888,15813.56,96.055953
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,2200.0,11.891892,11.000000,1,안국,0.002509,46623.003,3261,3261.00,130.888597
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2.01109e+07,근린상가,...,220.0,3.778770,3.778770,1,경복궁,0.004792,29040.490,2650,2650.00,10.823416
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2.0111e+07,근린상가,...,120.0,2.037351,1.812689,1,종로5가,0.003819,34219.916,209,390.00,39.418864


In [57]:
# 지하철거리 m로 변경
seoul_data_copy = seoul_data_copy.rename(columns = {'가까운지하철거리' : '지하철거리m'})
seoul_data_copy['지하철거리m'] = seoul_data_copy['지하철거리m'] * 1000

seoul_data_copy.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,RENT,rent_ua,rent_ca,floor,전철역명,지하철거리m,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리m
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1.98212e+07,근린상가,...,750.0,48.387097,48.387097,1,혜화,1.082077,71533.898,11937,15813.56,45.543627
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2.00807e+07,기타상가,...,1355.0,13.155340,9.414953,1,혜화,1.373136,117064.164,57888,15813.56,96.055953
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2.00812e+07,근린상가,...,2200.0,11.891892,11.000000,1,안국,2.508637,46623.003,3261,3261.00,130.888597
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2.01109e+07,근린상가,...,220.0,3.778770,3.778770,1,경복궁,4.791703,29040.490,2650,2650.00,10.823416
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2.0111e+07,근린상가,...,120.0,2.037351,1.812689,1,종로5가,3.818611,34219.916,209,390.00,39.418864


## 입주일 컬럼 이용해 건물 노후 정도 파악

In [58]:
# 입주일로 건물 노후 정도 파악

df2_seoul = seoul_data_copy.copy()

df2_seoul = df2_seoul[['상가코드', '건축물대장코드', '입력날짜', '도시', '구시군', '읍면동', '번', '지', '입주일',
       '상가유형', '지하총층', '지상총층', '대지면적m2', '건축면적m2', '연면적m2', '건폐율', '용적률',
       '건축구조', '높이m', '주차대수', '승강기수', '경도', '위도', 'n', 'usage_area',
       'cont_area', 'RENT', 'rent_ua', 'rent_ca', 'floor', '전철역명', '지하철거리m',
       '승객수점수', '건물별전기사용량', '본번별평균사용량', '횡단보도거리m']]

df2_seoul.isnull().sum()

상가코드          0
건축물대장코드       0
입력날짜          0
도시            0
구시군           0
읍면동           0
번             0
지             0
입주일           2
상가유형          0
지하총층          0
지상총층          0
대지면적m2        0
건축면적m2        0
연면적m2         0
건폐율           0
용적률           0
건축구조          0
높이m           0
주차대수          0
승강기수          0
경도            0
위도            0
n             0
usage_area    0
cont_area     0
RENT          0
rent_ua       0
rent_ca       0
floor         0
전철역명          0
지하철거리m        0
승객수점수         0
건물별전기사용량      0
본번별평균사용량      0
횡단보도거리m       0
dtype: int64

In [59]:
df2_seoul = df2_seoul.dropna()
df2_seoul.isnull().sum()

상가코드          0
건축물대장코드       0
입력날짜          0
도시            0
구시군           0
읍면동           0
번             0
지             0
입주일           0
상가유형          0
지하총층          0
지상총층          0
대지면적m2        0
건축면적m2        0
연면적m2         0
건폐율           0
용적률           0
건축구조          0
높이m           0
주차대수          0
승강기수          0
경도            0
위도            0
n             0
usage_area    0
cont_area     0
RENT          0
rent_ua       0
rent_ca       0
floor         0
전철역명          0
지하철거리m        0
승객수점수         0
건물별전기사용량      0
본번별평균사용량      0
횡단보도거리m       0
dtype: int64

In [60]:
df2_seoul[df2_seoul['입주일'] < 10000000]

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,RENT,rent_ua,rent_ca,floor,전철역명,지하철거리m,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리m
294,S11140000018240,11140-24326,202006,서울특별시,중구,서소문동,120,35,1970,근린상가,...,485.0,9.79798,2.866600,1,시청,3.480040,55399.652,19572,31409.77,29.267078
6500,S11650000008456,11650-14337,202006,서울특별시,서초구,서초동,1337,17,1980,기타상가,...,9670.0,4.17206,2.917211,1,강남,4.787219,200680.566,50472,91830.50,24.686496


In [61]:
# 입주일 오류 들어있는 행 임의로 변경
df2_seoul['입주일'] = df2_seoul['입주일'].astype('int')

df2_seoul.iloc[198,8] = 19801101
df2_seoul.iloc[245,8] = 19870901
df2_seoul.iloc[252,8] = 19850901
df2_seoul.iloc[268,8] = 19640701

In [62]:
df2_seoul = df2_seoul.reset_index()

df2_seoul = df2_seoul[['상가코드', '건축물대장코드', '입력날짜', '도시', '구시군', '읍면동', '번', '지', '입주일',
       '상가유형', '지하총층', '지상총층', '대지면적m2', '건축면적m2', '연면적m2', '건폐율', '용적률',
       '건축구조', '높이m', '주차대수', '승강기수', '경도', '위도', 'n', 'usage_area',
       'cont_area', 'RENT', 'rent_ua', 'rent_ca', 'floor', '전철역명', '지하철거리m',
       '승객수점수', '건물별전기사용량', '본번별평균사용량', '횡단보도거리m']]

In [63]:
df2_seoul.iloc[294, 8] = 19700101   # 월/일 없는 데이터 임의로 1월 1일로 넣기
df2_seoul.iloc[6498, 8] = 19800101  # 월/일 없는 데이터 임의로 1월 1일로 넣기

df2_seoul.iloc[468,8] = 19710930
df2_seoul.iloc[1816,8] = 19841101
df2_seoul.iloc[5127,8] = 19630401
df2_seoul.iloc[5890,8] = 19700226

In [64]:
df2_seoul['입주일'] = df2_seoul['입주일'].astype('object')

In [65]:
df2_seoul['입주일'] = pd.to_datetime(df2_seoul['입주일'], format = "%Y%m%d")

df2_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 36 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   상가코드        9340 non-null   object        
 1   건축물대장코드     9340 non-null   object        
 2   입력날짜        9340 non-null   int64         
 3   도시          9340 non-null   object        
 4   구시군         9340 non-null   object        
 5   읍면동         9340 non-null   object        
 6   번           9340 non-null   int64         
 7   지           9340 non-null   int64         
 8   입주일         9340 non-null   datetime64[ns]
 9   상가유형        9340 non-null   object        
 10  지하총층        9340 non-null   int64         
 11  지상총층        9340 non-null   int64         
 12  대지면적m2      9340 non-null   float64       
 13  건축면적m2      9340 non-null   float64       
 14  연면적m2       9340 non-null   float64       
 15  건폐율         9340 non-null   float64       
 16  용적률         9340 non-nul

In [66]:
df2_seoul['건물연차'] = df2_seoul['입주일'].apply(lambda x : round(((pd.datetime.today() - x).days)/365,0))
df2_seoul['건물연차'] = df2_seoul['건물연차'].astype('int')

df2_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   상가코드        9340 non-null   object        
 1   건축물대장코드     9340 non-null   object        
 2   입력날짜        9340 non-null   int64         
 3   도시          9340 non-null   object        
 4   구시군         9340 non-null   object        
 5   읍면동         9340 non-null   object        
 6   번           9340 non-null   int64         
 7   지           9340 non-null   int64         
 8   입주일         9340 non-null   datetime64[ns]
 9   상가유형        9340 non-null   object        
 10  지하총층        9340 non-null   int64         
 11  지상총층        9340 non-null   int64         
 12  대지면적m2      9340 non-null   float64       
 13  건축면적m2      9340 non-null   float64       
 14  연면적m2       9340 non-null   float64       
 15  건폐율         9340 non-null   float64       
 16  용적률         9340 non-nul

<ipython-input-66-1c7b5653700f>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df2_seoul['건물연차'] = df2_seoul['입주일'].apply(lambda x : round(((pd.datetime.today() - x).days)/365,0))


## 공시지가 컬럼 추가

In [67]:
gongsi = pd.read_csv("C:/data/서울특별시_개별공시지가정보_202010131637.csv", sep = '|')

gongsi

,고유번호,법정동코드,법정동명,지번,공시지가,표준지여부
0,1111013600100810000,1111013600,서울특별시 종로구 인사동,81,23690000,N
1,1111013600100830000,1111013600,서울특별시 종로구 인사동,83,23690000,N
2,1111013600100830001,1111013600,서울특별시 종로구 인사동,83-1,22490000,N
3,1111013600100840000,1111013600,서울특별시 종로구 인사동,84,20370000,N
4,1111013600100840002,1111013600,서울특별시 종로구 인사동,84-2,12610000,N
...,...,...,...,...,...,...
916373,1174010900103210015,1174010900,서울특별시 강동구 천호동,321-15,3396000,N
916374,1174010900103210016,1174010900,서울특별시 강동구 천호동,321-16,3563000,N
916375,1174010900103210017,1174010900,서울특별시 강동구 천호동,321-17,976800,N
916376,1174010900103210018,1174010900,서울특별시 강동구 천호동,321-18,3563000,N


In [68]:
seoul_data_copy = df2_seoul.copy()

seoul_data_copy.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,rent_ua,rent_ca,floor,전철역명,지하철거리m,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리m,건물연차
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1982-12-24,근린상가,...,48.387097,48.387097,1,혜화,1.082077,71533.898,11937,15813.56,45.543627,38
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2008-07-03,기타상가,...,13.155340,9.414953,1,혜화,1.373136,117064.164,57888,15813.56,96.055953,12
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2008-12-29,근린상가,...,11.891892,11.000000,1,안국,2.508637,46623.003,3261,3261.00,130.888597,12
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2011-09-28,근린상가,...,3.778770,3.778770,1,경복궁,4.791703,29040.490,2650,2650.00,10.823416,9
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2011-10-14,근린상가,...,2.037351,1.812689,1,종로5가,3.818611,34219.916,209,390.00,39.418864,9


In [69]:
aa = []
bb = []
cc = []

for i in gongsi['법정동명'].str.split(' '):
    aa.append(i[0])
    bb.append(i[1])
    cc.append(i[2])
    
gongsi['도시'] = aa
gongsi['구시군'] = bb
gongsi['읍면동'] = cc

gongsi.head(5)

,고유번호,법정동코드,법정동명,지번,공시지가,표준지여부,도시,구시군,읍면동
0,1111013600100810000,1111013600,서울특별시 종로구 인사동,81,23690000,N,서울특별시,종로구,인사동
1,1111013600100830000,1111013600,서울특별시 종로구 인사동,83,23690000,N,서울특별시,종로구,인사동
2,1111013600100830001,1111013600,서울특별시 종로구 인사동,83-1,22490000,N,서울특별시,종로구,인사동
3,1111013600100840000,1111013600,서울특별시 종로구 인사동,84,20370000,N,서울특별시,종로구,인사동
4,1111013600100840002,1111013600,서울특별시 종로구 인사동,84-2,12610000,N,서울특별시,종로구,인사동


In [70]:
gongsi = gongsi[['고유번호','법정동코드','도시','구시군','읍면동','지번','공시지가','표준지여부']]

In [71]:
aa = []
bb = []

for i in gongsi['지번'].str.split('-'):
    if len(i) < 2:
        aa.append(i[0])
        bb.append(0)
    else:
        aa.append(i[0])
        bb.append(i[1])
        
gongsi['번'] = aa
gongsi['지'] = bb

gongsi.head(5)

,고유번호,법정동코드,도시,구시군,읍면동,지번,공시지가,표준지여부,번,지
0,1111013600100810000,1111013600,서울특별시,종로구,인사동,81,23690000,N,81,0
1,1111013600100830000,1111013600,서울특별시,종로구,인사동,83,23690000,N,83,0
2,1111013600100830001,1111013600,서울특별시,종로구,인사동,83-1,22490000,N,83,1
3,1111013600100840000,1111013600,서울특별시,종로구,인사동,84,20370000,N,84,0
4,1111013600100840002,1111013600,서울특별시,종로구,인사동,84-2,12610000,N,84,2


In [72]:
seoul_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   상가코드        9340 non-null   object        
 1   건축물대장코드     9340 non-null   object        
 2   입력날짜        9340 non-null   int64         
 3   도시          9340 non-null   object        
 4   구시군         9340 non-null   object        
 5   읍면동         9340 non-null   object        
 6   번           9340 non-null   int64         
 7   지           9340 non-null   int64         
 8   입주일         9340 non-null   datetime64[ns]
 9   상가유형        9340 non-null   object        
 10  지하총층        9340 non-null   int64         
 11  지상총층        9340 non-null   int64         
 12  대지면적m2      9340 non-null   float64       
 13  건축면적m2      9340 non-null   float64       
 14  연면적m2       9340 non-null   float64       
 15  건폐율         9340 non-null   float64       
 16  용적률         9340 non-nul

In [73]:
gongsi_df = gongsi[['도시','구시군','읍면동','번','지','공시지가']]

gongsi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916378 entries, 0 to 916377
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   도시      916378 non-null  object
 1   구시군     916378 non-null  object
 2   읍면동     916378 non-null  object
 3   번       916378 non-null  object
 4   지       916378 non-null  object
 5   공시지가    916378 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 41.9+ MB


In [74]:
gongsi_df['번'] = gongsi_df['번'].astype('int64')
gongsi_df['지'] = gongsi_df['지'].astype('int64')

<ipython-input-74-f51342b8bd4f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongsi_df['번'] = gongsi_df['번'].astype('int64')
<ipython-input-74-f51342b8bd4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongsi_df['지'] = gongsi_df['지'].astype('int64')


In [75]:
gongsi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916378 entries, 0 to 916377
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   도시      916378 non-null  object
 1   구시군     916378 non-null  object
 2   읍면동     916378 non-null  object
 3   번       916378 non-null  int64 
 4   지       916378 non-null  int64 
 5   공시지가    916378 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 41.9+ MB


In [76]:
seoul_with_gongsi = pd.merge(seoul_data_copy, gongsi_df, on = ['도시','구시군','읍면동','번','지'], how = 'inner')

seoul_with_gongsi

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,rent_ca,floor,전철역명,지하철거리m,승객수점수,건물별전기사용량,본번별평균사용량,횡단보도거리m,건물연차,공시지가
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1982-12-24,근린상가,...,48.387097,1,혜화,1.082077,71533.898,11937,15813.56,45.543627,38,22840000
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2008-07-03,기타상가,...,9.414953,1,혜화,1.373136,117064.164,57888,15813.56,96.055953,12,16310000
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2008-12-29,근린상가,...,11.000000,1,안국,2.508637,46623.003,3261,3261.00,130.888597,12,24600000
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2011-09-28,근린상가,...,3.778770,1,경복궁,4.791703,29040.490,2650,2650.00,10.823416,9,7727000
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2011-10-14,근린상가,...,1.812689,1,종로5가,3.818611,34219.916,209,390.00,39.418864,9,4878000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9368,S11740000017328,11740-9659,202006,서울특별시,강동구,고덕동,645,4,1988-09-16,근린상가,...,1.894980,1,고덕,4.357042,8921.374,2408,3264.43,16.357351,32,5684000
9369,S11740000017337,11740-9676,202006,서울특별시,강동구,명일동,312,42,1980-06-27,근린상가,...,10.439427,1,명일,0.729846,33569.452,18399,12269.25,23.462974,40,12580000
9370,S11740000017364,11740-9698,202006,서울특별시,강동구,명일동,326,11,1981-10-05,근린상가,...,3.761755,1,명일,2.046134,16942.142,14429,8845.75,97.750285,39,6444000
9371,S11740000017467,11740-9843,202006,서울특별시,강동구,둔촌동,609,3,1993-12-06,근린상가,...,4.225308,1,길동,6.165909,22247.037,4695,9900.00,21.561184,27,6762000


In [77]:
seoul_with_gongsi.duplicated().value_counts()

False    9296
True       77
dtype: int64

In [78]:
seoul_with_gongsi = seoul_with_gongsi.drop_duplicates()

## 가장 가까운 아파트의 세대수와 평균평당가 컬럼 생성

In [79]:
# 구별 아파트 데이터 합치기

import pandas as pd  
import numpy as np  
import glob  
import sys

#파일 Union  
all_data = pd.DataFrame()  
for f in glob.glob('C:/data/seoul_apt_gu/구별아파트 (*).xlsx'): # 예를들어 201901, 201902 로 된 파일이면 2019_*  
    x = pd.read_excel(f)  
    all_data = pd.concat([all_data, x])

apt_data = all_data.copy()

apt_data = apt_data[['구시군','읍면동','아파트','세대수','3.3㎡당가(만원)']]
apt_data = apt_data.rename(columns = {'3.3㎡당가(만원)' : '평당가'})

apt_data = apt_data.dropna()

apt_data['평당가'] = apt_data['평당가'].astype('int')

apt_data

,구시군,읍면동,아파트,세대수,평당가
0,성북구,길음동,길음뉴타운11단지롯데캐슬골든힐스,81,2539
1,성북구,길음동,길음뉴타운11단지롯데캐슬골든힐스,14,2549
2,성북구,길음동,길음뉴타운11단지롯데캐슬골든힐스,35,2565
3,성북구,길음동,길음뉴타운11단지롯데캐슬골든힐스,6,2549
4,성북구,길음동,길음뉴타운11단지롯데캐슬골든힐스,34,2188
...,...,...,...,...,...
1275,중랑구,중화동,한신1차,144,1821
1276,중랑구,중화동,한신1차,54,1792
1277,중랑구,중화동,한신1차,104,1775
1278,중랑구,중화동,한신2차,86,2400


In [80]:
# 아파트별로 평당가 평균

import sqlite3
conn = sqlite3.connect(':memory:')

c = conn.cursor()

apt_data.to_sql('apt_data',conn,index = False)

c.execute('select * from apt_data')
c.fetchone()

('성북구', '길음동', '길음뉴타운11단지롯데캐슬골든힐스', 81, 2539)

In [81]:
c.execute("""select 구시군, 읍면동, 아파트, sum(세대수)
          from apt_data
          group by 읍면동, 아파트""")
table = c.fetchall()



c.execute("""select 구시군, 읍면동, 아파트, round(avg(평당가),2)
          from apt_data
          group by 읍면동, 아파트""")
table2 = c.fetchall()

c.close()
conn.close()

x = pd.DataFrame(table, columns = ['구시군','읍면동','아파트','총세대수'])
y = pd.DataFrame(table2, columns = ['구시군','읍면동','아파트','평균평당가'])

x_y = pd.merge(x, y, on = ['구시군','읍면동','아파트'])
x_y.isnull().sum()


x_y.head(10)

,구시군,읍면동,아파트,총세대수,평균평당가
0,송파구,가락동,SK허브파크,64,2157.00
1,송파구,가락동,가락금호,915,3779.00
2,송파구,가락동,가락동부센트레빌,264,2838.00
3,송파구,가락동,가락두산위브센티엄,61,1510.82
4,송파구,가락동,가락스타클래스,72,2280.57
5,송파구,가락동,"가락쌍용스윗닷홈3차(101,102동)",99,2755.00
6,송파구,가락동,가락쌍용스윗닷홈3차(103동),64,2344.00
7,송파구,가락동,가락쌍용스윗닷홈3차(104동),36,1818.00
8,송파구,가락동,가락쌍용스윗닷홈3차(105동),50,2136.00
9,송파구,가락동,가락한신,256,2651.50


In [82]:
# x_y 데이터 + apt_add 데이터 합치기

apt_add = pd.read_csv("C:/data/apt_ysd_add.csv", sep = '|')
apt_add2 = apt_add[apt_add['도시'] == '서울특별시'][['아파트명','구시군','읍면동','번지']]

xy_copy = x_y.copy()

apt_all = pd.merge(xy_copy, apt_add2, left_on = ['아파트','구시군','읍면동'], right_on = ['아파트명','구시군','읍면동'])
apt_all = apt_all.drop_duplicates()
apt_all = apt_all[['구시군','읍면동','번지','아파트','총세대수','평균평당가']]

apt_all

,구시군,읍면동,번지,아파트,총세대수,평균평당가
0,송파구,가락동,166-2,SK허브파크,64,2157.00
1,송파구,가락동,95-1,가락금호,915,3779.00
2,송파구,가락동,95,가락동부센트레빌,264,2838.00
3,송파구,가락동,98,가락두산위브센티엄,61,1510.82
4,송파구,가락동,8,가락스타클래스,72,2280.57
...,...,...,...,...,...,...
3600,동작구,흑석동,332,해가든,116,2298.86
3601,동작구,흑석동,335,흑석한강센트레빌,541,3949.86
3602,동작구,흑석동,339,흑석한강센트레빌2차,798,3696.23
3603,동작구,흑석동,336,흑석한강푸르지오,716,3610.33


In [83]:
# geocoding 이용해서 위/경도 뽑아오기

apt_bunji = apt_all[['구시군','읍면동','번지','아파트']]
apt_bunji['도시'] = '서울특별시'


import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()

apt_bunji.to_sql('apt_bunji', conn, index = False)

c.execute('select * from apt_bunji')
c.fetchone()

<ipython-input-83-516e2dd59cfc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apt_bunji['도시'] = '서울특별시'


('송파구', '가락동', '166-2', 'SK허브파크', '서울특별시')

In [84]:
c.execute("select 도시||' '||구시군||' '||읍면동||' '||번지, 아파트 from apt_bunji")
table = c.fetchall()

bj = pd.DataFrame(table)
bj

# bj.to_csv("C:/data/now_used/bj.csv", index = False)

c.close()
conn.close()

In [85]:
# qgis를 이용해 csv로 변환 후 다운로드

bj_long = pd.read_csv("C:/data/now_used/bj_long.csv", encoding = 'ANSI')
bj_long.columns

bj_long = bj_long[['주소', '아파트명', 'X', 'Y']]

bj_long['주소'].str.split(' ')

0       [서울특별시, 송파구, 가락동, 166-2]
1        [서울특별시, 송파구, 가락동, 95-1]
2          [서울특별시, 송파구, 가락동, 95]
3          [서울특별시, 송파구, 가락동, 98]
4           [서울특별시, 송파구, 가락동, 8]
                  ...           
3487      [서울특별시, 동작구, 흑석동, 332]
3488      [서울특별시, 동작구, 흑석동, 335]
3489      [서울특별시, 동작구, 흑석동, 339]
3490      [서울특별시, 동작구, 흑석동, 336]
3491       [서울특별시, 중구, 흥인동, 3-4]
Name: 주소, Length: 3492, dtype: object

In [86]:
aa = []
bb = []
cc = []
dd = []

for i in bj_long['주소'].str.split(' '):
    aa.append(i[0])
    bb.append(i[1])
    cc.append(i[2])
    dd.append(i[3])

bj_long['도시'] = aa
bj_long['구시군'] = bb
bj_long['읍면동'] = cc
bj_long['번지'] = dd

bj_long.head(5)

,주소,아파트명,X,Y,도시,구시군,읍면동,번지
0,서울특별시 송파구 가락동 166-2,SK허브파크,127.134231,37.497960,서울특별시,송파구,가락동,166-2
1,서울특별시 송파구 가락동 95-1,가락금호,127.116383,37.496206,서울특별시,송파구,가락동,95-1
2,서울특별시 송파구 가락동 95,가락동부센트레빌,127.116376,37.497773,서울특별시,송파구,가락동,95
3,서울특별시 송파구 가락동 98,가락두산위브센티엄,127.119128,37.494514,서울특별시,송파구,가락동,98
4,서울특별시 송파구 가락동 8,가락스타클래스,127.126088,37.502820,서울특별시,송파구,가락동,8


In [87]:
bj_long = bj_long[['아파트명','구시군','읍면동','번지','X','Y']]
bj_long = bj_long.rename(columns = {'X' : '경도', 'Y' : '위도'})

bj_long.columns

Index(['아파트명', '구시군', '읍면동', '번지', '경도', '위도'], dtype='object')

In [88]:
apt_all.columns

Index(['구시군', '읍면동', '번지', '아파트', '총세대수', '평균평당가'], dtype='object')

In [89]:
# bj_long + apt_all

df_apt = pd.merge(apt_all, bj_long[['구시군','읍면동','아파트명','경도','위도']], left_on = ['구시군','읍면동','아파트'], right_on = ['구시군','읍면동','아파트명'], how = 'left')[['구시군','읍면동','아파트','총세대수','평균평당가','경도','위도']]
df_apt = df_apt.drop_duplicates()

df_apt.head(5)

,구시군,읍면동,아파트,총세대수,평균평당가,경도,위도
0,송파구,가락동,SK허브파크,64,2157.00,127.134231,37.497960
1,송파구,가락동,가락금호,915,3779.00,127.116383,37.496206
2,송파구,가락동,가락동부센트레빌,264,2838.00,127.116376,37.497773
3,송파구,가락동,가락두산위브센티엄,61,1510.82,127.119128,37.494514
4,송파구,가락동,가락스타클래스,72,2280.57,127.126088,37.502820


In [90]:
import sqlite3
conn = sqlite3.connect(':memory:')

c = conn.cursor()

df_apt.to_sql('df_apt',conn,index = False)

c.execute('select * from df_apt')
c.fetchone()

('송파구', '가락동', 'SK허브파크', 64, 2157.0, 127.134231, 37.49796)

In [91]:
c.execute("""select 구시군||' '||읍면동||' '||아파트, 경도, 위도
          from df_apt""")
table = c.fetchall()

len(table)

df_apt_add = pd.DataFrame(table, columns = ['아파트','경도','위도'])
df_apt_add

c.close()
conn.close()

# df_apt_add.to_csv("C:/data/now_used/apartments.csv", index = False)

In [94]:
# 상가-아파트 3개 데이터에 아파트 세대수, 평균평당가 컬럼 추가
apt_qgis = pd.read_csv("C:/data/now_used/거리행렬.csv")

apt_qgis.head(5)

,InputID,TargetID,Distance
0,11110-10014,종로구 명륜2가 아남3차,519.940374
1,11110-10014,종로구 명륜2가 아남,539.272823
2,11110-10014,종로구 명륜2가 아남2차,613.099144
3,11110-100180009,종로구 명륜2가 아남3차,497.856179
4,11110-100180009,종로구 명륜2가 아남,527.971635


In [95]:
aa = []
bb = []
cc = []

for i in apt_qgis['TargetID'].str.split(' '):
    aa.append(i[0])
    bb.append(i[1])
    cc.append(i[2])

apt_qgis['구시군'] = aa
apt_qgis['읍면동'] = bb
apt_qgis['아파트'] = cc

apt_qgis

,InputID,TargetID,Distance,구시군,읍면동,아파트
0,11110-10014,종로구 명륜2가 아남3차,519.940374,종로구,명륜2가,아남3차
1,11110-10014,종로구 명륜2가 아남,539.272823,종로구,명륜2가,아남
2,11110-10014,종로구 명륜2가 아남2차,613.099144,종로구,명륜2가,아남2차
3,11110-100180009,종로구 명륜2가 아남3차,497.856179,종로구,명륜2가,아남3차
4,11110-100180009,종로구 명륜2가 아남,527.971635,종로구,명륜2가,아남
...,...,...,...,...,...,...
27951,11740-9843,강동구 길동 하나플러스,179.564264,강동구,길동,하나플러스
27952,11740-9843,강동구 둔촌동 초원동아(621),197.561971,강동구,둔촌동,초원동아(621)
27953,11740-9917,강동구 길동 신암가온,57.213374,강동구,길동,신암가온
27954,11740-9917,강동구 길동 다성이즈빌(388-4),103.200113,강동구,길동,다성이즈빌(388-4)


In [98]:
apt_qgis.duplicated().value_counts()

False    27854
True       102
dtype: int64

In [96]:
apt_qgis = apt_qgis[['InputID','구시군','읍면동','아파트','Distance']]

apt_merge = pd.merge(apt_qgis, df_apt[['구시군','읍면동','아파트','총세대수','평균평당가']], on = ['구시군','읍면동','아파트'])
apt_merge

,InputID,구시군,읍면동,아파트,Distance,총세대수,평균평당가
0,11110-10014,종로구,명륜2가,아남3차,519.940374,136,1506.40
1,11110-100180009,종로구,명륜2가,아남3차,497.856179,136,1506.40
2,11110-100195520,종로구,명륜2가,아남3차,597.274432,136,1506.40
3,11110-11314,종로구,명륜2가,아남3차,423.295367,136,1506.40
4,11110-12178,종로구,명륜2가,아남3차,700.505548,136,1506.40
...,...,...,...,...,...,...,...
28156,11740-7485,강동구,천호동,현대코아,42.152575,185,921.71
28157,11740-7335,강동구,암사동,포스파크,230.229833,134,1352.75
28158,11740-8604,강동구,성내동,성내3차e편한세상,76.437020,62,2134.00
28159,11740-9253,강동구,암사동,암사한솔,192.664473,203,1874.00


In [99]:
apt_merge = apt_merge.drop_duplicates()

In [102]:
apt_merge['세대수점수'] = round((1/(apt_merge['Distance'])) * apt_merge['총세대수'],3)

apt_merge.head(5)

<ipython-input-102-4784e0efc1b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apt_merge['세대수점수'] = round((1/(apt_merge['Distance'])) * apt_merge['총세대수'],3)


,InputID,구시군,읍면동,아파트,Distance,총세대수,평균평당가,세대수점수
0,11110-10014,종로구,명륜2가,아남3차,519.940374,136,1506.4,0.262
1,11110-100180009,종로구,명륜2가,아남3차,497.856179,136,1506.4,0.273
2,11110-100195520,종로구,명륜2가,아남3차,597.274432,136,1506.4,0.228
3,11110-11314,종로구,명륜2가,아남3차,423.295367,136,1506.4,0.321
4,11110-12178,종로구,명륜2가,아남3차,700.505548,136,1506.4,0.194


In [103]:
apt_merge['평당가점수'] = round((1/(apt_merge['Distance'])) * apt_merge['평균평당가'],3)

apt_merge.head(5)

<ipython-input-103-8ee651cdcf7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apt_merge['평당가점수'] = round((1/(apt_merge['Distance'])) * apt_merge['평균평당가'],3)


,InputID,구시군,읍면동,아파트,Distance,총세대수,평균평당가,세대수점수,평당가점수
0,11110-10014,종로구,명륜2가,아남3차,519.940374,136,1506.4,0.262,2.897
1,11110-100180009,종로구,명륜2가,아남3차,497.856179,136,1506.4,0.273,3.026
2,11110-100195520,종로구,명륜2가,아남3차,597.274432,136,1506.4,0.228,2.522
3,11110-11314,종로구,명륜2가,아남3차,423.295367,136,1506.4,0.321,3.559
4,11110-12178,종로구,명륜2가,아남3차,700.505548,136,1506.4,0.194,2.150


In [126]:
import sqlite3

conn = sqlite3.connect(':memory:')
c = conn.cursor()

apt_merge.to_sql('apt_merge',conn,index = False)

c.execute('select * from apt_merge')
c.fetchone()

('11110-10014',
 '종로구',
 '명륜2가',
 '아남3차',
 519.940373679,
 136,
 1506.4,
 0.262,
 2.897)

In [127]:
c.execute("""select InputID, 구시군, 읍면동, 아파트, 총세대수, 평균평당가, Distance, sum(세대수점수), sum(평당가점수)
          from apt_merge
          group by InputID""")
table = c.fetchall()

c.close()
conn.close()

apt_df = pd.DataFrame(table, columns = ['건축물대장코드','구시군','읍면동','가까운아파트','가까운아파트총세대수','가까운아파트평균평당가','아파트거리','주변세대수점수','주변평당가점수'])
apt_df

,건축물대장코드,구시군,읍면동,가까운아파트,가까운아파트총세대수,가까운아파트평균평당가,아파트거리,주변세대수점수,주변평당가점수
0,11110-10014,종로구,명륜2가,아남3차,136,1506.40,519.940374,1.176,10.477
1,11110-100180009,종로구,명륜2가,아남3차,136,1506.40,497.856179,1.207,10.754
2,11110-100184402,종로구,견지동,대성스카이렉스,54,2008.50,249.441747,0.363,19.104
3,11110-100190514,종로구,사직동,광화문Space本1단지,647,2695.24,426.832578,1.979,16.353
4,11110-100190842,종로구,인의동,효성주얼리시티,282,2046.33,491.694446,0.810,10.063
...,...,...,...,...,...,...,...,...,...
9257,11740-9659,강동구,고덕동,고덕아이파크,1142,3703.89,244.901088,18.968,40.630
9258,11740-9676,강동구,명일동,중앙,47,1166.60,275.096339,12.853,39.510
9259,11740-9698,강동구,명일동,중앙,47,1166.60,14.138738,10.374,109.826
9260,11740-9843,강동구,길동,하나플러스,68,1666.00,179.564264,7.339,47.818


In [135]:
# 상가테이블과 merge
real_final_data = pd.merge(seoul_with_gongsi, apt_df[['건축물대장코드','가까운아파트','가까운아파트총세대수','가까운아파트평균평당가','아파트거리','주변세대수점수','주변평당가점수']], on = '건축물대장코드')

real_final_data.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,본번별평균사용량,횡단보도거리m,건물연차,공시지가,가까운아파트,가까운아파트총세대수,가까운아파트평균평당가,아파트거리,주변세대수점수,주변평당가점수
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1982-12-24,근린상가,...,15813.56,45.543627,38,22840000,아남3차,136,1506.40,519.940374,1.176,10.477
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2008-07-03,기타상가,...,15813.56,96.055953,12,16310000,아남3차,136,1506.40,497.856179,1.207,10.754
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2008-12-29,근린상가,...,3261.00,130.888597,12,24600000,대성스카이렉스,54,2008.50,249.441747,0.363,19.104
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2011-09-28,근린상가,...,2650.00,10.823416,9,7727000,광화문Space本1단지,647,2695.24,426.832578,1.979,16.353
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2011-10-14,근린상가,...,390.00,39.418864,9,4878000,효성주얼리시티,282,2046.33,491.694446,0.810,10.063


In [136]:
real_final_data.columns

Index(['상가코드', '건축물대장코드', '입력날짜', '도시', '구시군', '읍면동', '번', '지', '입주일', '상가유형',
       '지하총층', '지상총층', '대지면적m2', '건축면적m2', '연면적m2', '건폐율', '용적률', '건축구조',
       '높이m', '주차대수', '승강기수', '경도', '위도', 'n', 'usage_area', 'cont_area',
       'RENT', 'rent_ua', 'rent_ca', 'floor', '전철역명', '지하철거리m', '승객수점수',
       '건물별전기사용량', '본번별평균사용량', '횡단보도거리m', '건물연차', '공시지가', '가까운아파트',
       '가까운아파트총세대수', '가까운아파트평균평당가', '아파트거리', '주변세대수점수', '주변평당가점수'],
      dtype='object')

In [137]:
real_final_data.to_csv("C:/data/now_used/real_final_data_all.csv", index = False)

## 더미변수 만들기

In [129]:
from pandas import get_dummies

type_dummies = pd.get_dummies(real_final_data['상가유형'], prefix = 'type')

real_final_data = pd.concat([real_final_data, type_dummies], axis = 1)

lst = []
[lst.append(1) if i > 0 else lst.append(0) for i in real_final_data['주차대수']]

real_final_data['주차가능여부'] = lst

lst = []
[lst.append(1) if i > 0 else lst.append(0) for i in real_final_data['승강기수']]

real_final_data['승강기여부'] = lst


real_final_data['상가유형'].unique()
lst = []
for i in real_final_data['상가유형']:
    if i == '근린상가':
        lst.append(1)
    elif i == '기타상가':
        lst.append(2)
    elif i == '복합상가':
        lst.append(3)
    else:
        lst.append(4)

real_final_data['상가유형번호'] = lst

real_final_data.head(5)

,상가코드,건축물대장코드,입력날짜,도시,구시군,읍면동,번,지,입주일,상가유형,...,아파트거리,주변세대수점수,주변평당가점수,type_근린상가,type_기타상가,type_단지내상가,type_복합상가,주차가능여부,승강기여부,상가유형번호
0,S11110000000004,11110-10014,202006,서울특별시,종로구,동숭동,1,114,1982-12-24,근린상가,...,519.940374,1.176,10.477,1,0,0,0,0,1,1
1,S11110000000125,11110-100180009,202006,서울특별시,종로구,동숭동,1,67,2008-07-03,기타상가,...,497.856179,1.207,10.754,0,1,0,0,1,1,2
2,S11110000000220,11110-100184402,202006,서울특별시,종로구,관훈동,186,0,2008-12-29,근린상가,...,249.441747,0.363,19.104,1,0,0,0,0,1,1
3,S11110000000602,11110-100190514,202006,서울특별시,종로구,누하동,245,6,2011-09-28,근린상가,...,426.832578,1.979,16.353,1,0,0,0,1,0,1
4,S11110000000634,11110-100190842,202006,서울특별시,종로구,효제동,19,3,2011-10-14,근린상가,...,491.694446,0.810,10.063,1,0,0,0,0,0,1


In [130]:
real_final_data.columns

Index(['상가코드', '건축물대장코드', '입력날짜', '도시', '구시군', '읍면동', '번', '지', '입주일', '상가유형',
       '지하총층', '지상총층', '대지면적m2', '건축면적m2', '연면적m2', '건폐율', '용적률', '건축구조',
       '높이m', '주차대수', '승강기수', '경도', '위도', 'n', 'usage_area', 'cont_area',
       'RENT', 'rent_ua', 'rent_ca', 'floor', '전철역명', '지하철거리m', '승객수점수',
       '건물별전기사용량', '본번별평균사용량', '횡단보도거리m', '건물연차', '공시지가', '가까운아파트',
       '가까운아파트총세대수', '가까운아파트평균평당가', '아파트거리', '주변세대수점수', '주변평당가점수', 'type_근린상가',
       'type_기타상가', 'type_단지내상가', 'type_복합상가', '주차가능여부', '승강기여부', '상가유형번호'],
      dtype='object')

In [131]:
# 0값 많은 대지면적m2, 건축면적m2, 높이m, 건폐율, 용적률과 기타 object타입 컬럼 제거

real_final_data = real_final_data[['건축물대장코드', '도시', '구시군', '읍면동', '번', '지', '입주일', 
       '지하총층', '지상총층', '연면적m2','주차대수', '승강기수', 'usage_area', 'cont_area',
       'RENT', 'rent_ua', 'rent_ca', '지하철거리m', '승객수점수',
       '건물별전기사용량', '본번별평균사용량', '횡단보도거리m', '건물연차', '공시지가',
       '가까운아파트총세대수', '가까운아파트평균평당가', '아파트거리', '주변세대수점수', '주변평당가점수','type_근린상가', 'type_기타상가', 'type_단지내상가', 'type_복합상가',
       '주차가능여부', '승강기여부', '상가유형번호']]

real_final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9296 entries, 0 to 9295
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   건축물대장코드      9296 non-null   object        
 1   도시           9296 non-null   object        
 2   구시군          9296 non-null   object        
 3   읍면동          9296 non-null   object        
 4   번            9296 non-null   int64         
 5   지            9296 non-null   int64         
 6   입주일          9296 non-null   datetime64[ns]
 7   지하총층         9296 non-null   int64         
 8   지상총층         9296 non-null   int64         
 9   연면적m2        9296 non-null   float64       
 10  주차대수         9296 non-null   int64         
 11  승강기수         9296 non-null   int64         
 12  usage_area   9296 non-null   float64       
 13  cont_area    9296 non-null   float64       
 14  RENT         9296 non-null   float64       
 15  rent_ua      9296 non-null   float64       
 16  rent_c

In [132]:
# 연면적m2 0인 행만 제거
real_final_data = real_final_data[real_final_data.연면적m2 != 0]

real_final_data

,건축물대장코드,도시,구시군,읍면동,번,지,입주일,지하총층,지상총층,연면적m2,...,아파트거리,주변세대수점수,주변평당가점수,type_근린상가,type_기타상가,type_단지내상가,type_복합상가,주차가능여부,승강기여부,상가유형번호
0,11110-10014,서울특별시,종로구,동숭동,1,114,1982-12-24,1,5,1563.66,...,519.940374,1.176,10.477,1,0,0,0,0,1,1
1,11110-100180009,서울특별시,종로구,동숭동,1,67,2008-07-03,4,5,8585.08,...,497.856179,1.207,10.754,0,1,0,0,1,1,2
2,11110-100184402,서울특별시,종로구,관훈동,186,0,2008-12-29,0,3,172.11,...,249.441747,0.363,19.104,1,0,0,0,0,1,1
3,11110-100190514,서울특별시,종로구,누하동,245,6,2011-09-28,0,4,233.03,...,426.832578,1.979,16.353,1,0,0,0,1,0,1
4,11110-100190842,서울특별시,종로구,효제동,19,3,2011-10-14,0,5,253.80,...,491.694446,0.810,10.063,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9291,11740-9659,서울특별시,강동구,고덕동,645,4,1988-09-16,1,4,493.71,...,244.901088,18.968,40.630,1,0,0,0,1,0,1
9292,11740-9676,서울특별시,강동구,명일동,312,42,1980-06-27,1,3,988.04,...,275.096339,12.853,39.510,1,0,0,0,0,0,1
9293,11740-9698,서울특별시,강동구,명일동,326,11,1981-10-05,1,2,741.99,...,14.138738,10.374,109.826,1,0,0,0,1,0,1
9294,11740-9843,서울특별시,강동구,둔촌동,609,3,1993-12-06,1,4,659.00,...,179.564264,7.339,47.818,1,0,0,0,1,0,1


In [133]:
real_final_data = real_final_data.reset_index()
real_final_data = real_final_data.iloc[:,1:]

real_final_data

,건축물대장코드,도시,구시군,읍면동,번,지,입주일,지하총층,지상총층,연면적m2,...,아파트거리,주변세대수점수,주변평당가점수,type_근린상가,type_기타상가,type_단지내상가,type_복합상가,주차가능여부,승강기여부,상가유형번호
0,11110-10014,서울특별시,종로구,동숭동,1,114,1982-12-24,1,5,1563.66,...,519.940374,1.176,10.477,1,0,0,0,0,1,1
1,11110-100180009,서울특별시,종로구,동숭동,1,67,2008-07-03,4,5,8585.08,...,497.856179,1.207,10.754,0,1,0,0,1,1,2
2,11110-100184402,서울특별시,종로구,관훈동,186,0,2008-12-29,0,3,172.11,...,249.441747,0.363,19.104,1,0,0,0,0,1,1
3,11110-100190514,서울특별시,종로구,누하동,245,6,2011-09-28,0,4,233.03,...,426.832578,1.979,16.353,1,0,0,0,1,0,1
4,11110-100190842,서울특별시,종로구,효제동,19,3,2011-10-14,0,5,253.80,...,491.694446,0.810,10.063,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9284,11740-9659,서울특별시,강동구,고덕동,645,4,1988-09-16,1,4,493.71,...,244.901088,18.968,40.630,1,0,0,0,1,0,1
9285,11740-9676,서울특별시,강동구,명일동,312,42,1980-06-27,1,3,988.04,...,275.096339,12.853,39.510,1,0,0,0,0,0,1
9286,11740-9698,서울특별시,강동구,명일동,326,11,1981-10-05,1,2,741.99,...,14.138738,10.374,109.826,1,0,0,0,1,0,1
9287,11740-9843,서울특별시,강동구,둔촌동,609,3,1993-12-06,1,4,659.00,...,179.564264,7.339,47.818,1,0,0,0,1,0,1


In [134]:
real_final_data.to_csv("C:/data/now_used/real_final_data.csv", index = False)